In [2]:
# import pandas as pd
# from_date = '2022-06-01'
# to_date = '2022-06-01'

# by_sale_obj = BySaleGroupGatepass.objects.filter(bill_date__date__range=[from_date, to_date])
# by_sale_values = list(by_sale_obj.values_list('by_sale_group__business__businessagentmap__agent__first_name', 'by_sale_group__business__businessagentmap__agent__agent_profile__gst_number', 'bill_number', 'bill_date', 'by_sale_group__total_cost', 'by_sale_group__bysale__by_product__cgst_percent', 'by_sale_group__bysale__by_product__sgst_percent', 'by_sale_group__bysale__cost', 'by_sale_group__bysale__cgst_value', 'by_sale_group__bysale__sgst_value', 'id'))
# by_sale_column = ['agent_name', 'gst_number', 'bill_no', 'bill_date', 'total_cost', 'csgt', 'sgst', 'taxable_value', 'cgst_value', 'sgst_value', 'bill_id']
# by_sale_df = pd.DataFrame(by_sale_values, columns=by_sale_column)
# by_sale_df['bill_date'] = by_sale_df['bill_date'].dt.strftime('%d-%b-%Y')
# by_sale_df['total_gst'] = by_sale_df['csgt'] + by_sale_df['sgst']
# by_sale_df = by_sale_df.fillna('')
# by_sale_df = by_sale_df.groupby(['bill_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_no': 'first', 'bill_date': 'first', 'total_cost': 'first','taxable_value': sum , 'cgst_value': sum, 'sgst_value': sum}).reset_index()

# final_df = by_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_no': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost': 'INVOICEVALUE', 'total_gst': 'RATE', 'taxable_value': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
# final_df['PLACEOFSUPPLY'] = ''
# final_df['REVERSECHARGE'] = ''
# final_df['INVOICETYPE'] = ''
# final_df['IGST'] = 0
# final_df = final_df.drop(columns=['bill_id'])
# final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]

# final_df

,business_id,bill_number
0,2614,19200
1,2873,19205
2,3110,19203
3,2659,19202
4,2628,19201


In [62]:
from decimal import Decimal
import numpy as np
from datetime import datetime
business_codes = list(Business.objects.filter(business_type_id=3).values_list('code', flat=True))
from_date = '2022-04-01'
to_date = '2022-04-10'

business_wise_bill = BusinessWiseBillNumber.objects.filter(business__code__in=business_codes, from_date=from_date, to_date=to_date)
business_wise_bill_list = list(business_wise_bill.values_list('business_id', 'bill_number'))
business_wise_bill_columns = ['business_id', 'bill_number']
business_wise_bill_df = pd.DataFrame(business_wise_bill_list, columns=business_wise_bill_columns)

date_in_format = datetime.strptime(str(to_date), '%Y-%m-%d')
date = datetime.strftime(date_in_format, '%d-%b-%Y')
product_quantity_check_dict = serve_qty_and_short_name(from_date, to_date)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes, sale_group__date__gte=from_date, sale_group__date__lte=to_date, product__group_id=2).exclude(product_id=28)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code', 'sale_group__business__businessagentmap__agent__first_name', 'sale_group__business__businessagentmap__agent__agent_profile__gst_number'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code', 'agent_name', 'gst_number']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first', 'agent_name': 'first', 'gst_number': 'first'}).reset_index()
short_name_and_qty = product_quantity_check_dict['product_dict']

if not sale_df.empty:
    sale_df['cost'] = sale_df['cost'].astype(float)
    sale_df['mrp'] = sale_df['cost'] / sale_df['count']
    sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
    sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
    sale_df['cost_by_base_price'] = sale_df.apply(lambda x: find_base_price_from_total_cost(x['cost'], x['cgst_percent'] + x['sgst_percent']), axis=1)
    sale_df['cost_by_base_price'] = sale_df['cost_by_base_price'].astype(float)
    sale_df['base_price'] = sale_df['cost_by_base_price'] / sale_df['count']
    sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
    sale_df['total_gst_for_base_price'] = sale_df['cost'] - sale_df['cost_by_base_price']
    sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
    sale_df['total_gst'] = sale_df['cgst_percent'] + sale_df['sgst_percent']
    sale_df['total_cost'] = (sale_df['cost_by_base_price'] + sale_df['total_gst_for_base_price'])

    sale_df = sale_df.merge(business_wise_bill_df, how="left", on="business_id").fillna("")
    sale_df = sale_df[sale_df['bill_number'] != ""]
# grouped_sale_df = sale_df.groupby(['business_id', 'total_gst']).agg({'agent_name': 'first','gst_number': 'first','bill_number': 'first', 'total_cost': 'sum','cost_by_base_price': sum , 'total_gst_for_base_price': sum}).reset_index()
# grouped_sale_df['total_cost_rounded'] = grouped_sale_df['total_cost'].apply(np.ceil)
# grouped_sale_df['cgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
# grouped_sale_df['sgst_value'] = grouped_sale_df['total_gst_for_base_price'] / 2
# grouped_sale_df['bill_date'] = date

# final_df = grouped_sale_df.rename(columns = {'gst_number': 'GSTIN_UINOFRECIPIENT', 'agent_name': 'NAME', 'bill_number': 'INVOICENO', 'bill_date' : 'INVOICEDATE', 'total_cost_rounded': 'INVOICEVALUE', 'total_gst': 'RATE', 'cost_by_base_price': 'TAXABLEVALUE', 'cgst_value': 'CGST', 'sgst_value': 'SGST'})
# final_df['PLACEOFSUPPLY'] = ''
# final_df['REVERSECHARGE'] = ''
# final_df['INVOICETYPE'] = ''
# final_df['IGST'] = 0
# final_df = final_df[['GSTIN_UINOFRECIPIENT', 'NAME', 'INVOICENO', 'INVOICEDATE', 'INVOICEVALUE', 'PLACEOFSUPPLY', 'REVERSECHARGE', 'INVOICETYPE', 'RATE', 'TAXABLEVALUE', 'CGST', 'SGST', 'IGST']]
sale_df

,business_id,product_id,count,user_profile_id,business_code,cgst_percent,sgst_percent,cost,hsn_code,agent_name,...,mrp,product_quantity,product_short_name,cost_by_base_price,base_price,total_gst_for_base_price,qty_in_whole,total_gst,total_cost,bill_number
0,1714,7,800.0,1725,2000,2.50,2.50,8000.0,0123,Union Booth,...,10.0,140.0,Curd 140,7619.05,9.523813,380.95,112.00,5.00,8000.0,15247
1,1714,25,1296.0,1725,2000,6.00,6.00,38880.0,0233,Union Booth,...,30.0,460.0,Curd 460,34714.29,26.785718,4165.71,596.16,12.00,38880.0,15247
2,1775,7,650.0,1786,8101,2.50,2.50,6500.0,0123,UNION PARLOUR,...,10.0,140.0,Curd 140,6190.48,9.523815,309.52,91.00,5.00,6500.0,15245
3,1775,25,1512.0,1786,8101,6.00,6.00,45360.0,0233,UNION PARLOUR,...,30.0,460.0,Curd 460,40500.00,26.785714,4860.00,695.52,12.00,45360.0,15245
4,1775,26,300.0,1786,8101,2.50,2.50,3600.0,,UNION PARLOUR,...,12.0,200.0,BMJar,3428.57,11.428567,171.43,60.00,5.00,3600.0,15245
5,2059,7,200.0,2070,8104,2.50,2.50,2000.0,0123,UKKADAM BUS STAND PARLOUR,...,10.0,140.0,Curd 140,1904.76,9.523800,95.24,28.00,5.00,2000.0,15244
6,2059,25,300.0,2070,8104,6.00,6.00,9000.0,0233,UKKADAM BUS STAND PARLOUR,...,30.0,460.0,Curd 460,8035.71,26.785700,964.29,138.00,12.00,9000.0,15244
7,2059,26,372.0,2070,8104,2.50,2.50,4464.0,,UKKADAM BUS STAND PARLOUR,...,12.0,200.0,BMJar,4251.43,11.428575,212.57,74.40,5.00,4464.0,15244
8,2060,7,470.0,2071,8115,2.50,2.50,4700.0,0123,HI-TECH PARLOUR,...,10.0,140.0,Curd 140,4476.19,9.523809,223.81,65.80,5.00,4700.0,15243
9,2060,25,730.0,2071,8115,6.00,6.00,21900.0,0233,HI-TECH PARLOUR,...,30.0,460.0,Curd 460,19553.57,26.785712,2346.43,335.80,12.00,21900.0,15243


In [58]:
product_wise_df = sale_df
product_wise_df['bill_date'] = date
product_wise_df['cgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['sgst_value'] = product_wise_df['total_gst_for_base_price'] / 2
product_wise_df['ISGT'] = 0.000
product_wise_df = product_wise_df.rename(columns={'agent_name': 'AGENT', 'gst_number': 'BTAXNO', 'product_short_name': 'PRODUCT', 'bill_number': 'INVNO', 'bill_date': 'INVDATE', 'sgst_value': 'SGST','cgst_value': 'CGST', 'total_cost': 'NETAMOUNT', 'hsn_code': 'HSNCODE', 'total_gst': 'GST', 'mrp': 'RATE'})
final_df_2 = product_wise_df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE']]
final_df_2



,AGENT,BTAXNO,PRODUCT,INVNO,INVDATE,RATE,GST,SGST,CGST,ISGT,NETAMOUNT,HSNCODE
0,Day to day Supermarket,0,Curd 460,19200,10-Apr-2022,24.78,12.00,292.050,292.050,0.0,5451.60,0233
1,abdu,0,Curd 460,19201,10-Apr-2022,24.78,12.00,219.040,219.040,0.0,4088.70,0233
4,Tamilselvi,0,Curd 140,19202,10-Apr-2022,8.00,5.00,38.095,38.095,0.0,1600.00,0123
5,Tamilselvi,0,Curd 460,19202,10-Apr-2022,24.78,12.00,306.650,306.650,0.0,5724.18,0233
6,A.Perumal Adaikkan,0,Curd 140,19205,10-Apr-2022,8.00,5.00,923.810,923.810,0.0,38800.00,0123
7,A.Perumal Adaikkan,0,Curd 460,19205,10-Apr-2022,24.78,12.00,4078.080,4078.080,0.0,76124.16,0233
8,KAILASH RAJA/ANNAI LPG STOVE SALES AND SERVICE,0,Curd 140,19203,10-Apr-2022,8.00,5.00,34.285,34.285,0.0,1440.00,0123
9,KAILASH RAJA/ANNAI LPG STOVE SALES AND SERVICE,0,Curd 460,19203,10-Apr-2022,24.78,12.00,398.250,398.250,0.0,7434.00,0233
10,KAILASH RAJA/ANNAI LPG STOVE SALES AND SERVICE,0,BMJar,19203,10-Apr-2022,9.00,5.00,33.430,33.430,0.0,1404.00,0


In [9]:
def find_base_price_from_total_cost(total_cost, total_gst):
    whole_gst = (total_gst / 100) + 1
    base_price = Decimal(total_cost) / whole_gst
    return round(base_price, 2)
def serve_qty_and_short_name(start_date, end_date):
    product_dict = {}
    for product in Product.objects.filter():
        if not product.id in product_dict:
            product_dict[product.id] = {}
        trace_ids = []
        for product_trace in ProductNameAndQuantityTrace.objects.filter(product_id=product.id):
            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
            if start_date in date_list :
                trace_ids.append(product_trace.id)
            if end_date in date_list:
                trace_ids.append(product_trace.id)
        if len(set(trace_ids)) == 1:
            product_trace_id = trace_ids[0]
            product_trace_obj = ProductNameAndQuantityTrace.objects.get(id=product_trace_id)
            data_dict = {
                'qty': product_trace_obj.quantity,
                'short_name': product_trace_obj.short_name
            }
            product_dict[product.id] = data_dict
        else:
            data_dict = {
                'is_multiple_qty': True,
            }
            return data_dict
    final_dict = {
        'is_multiple_qty' : False,
        'product_dict': product_dict
    }
    return final_dict

In [1]:
def to_find_values_in_kg_ltr(by_product_unit_name, group_unit, total_qty):
    value = 0
    if by_product_unit_name == 'Gr' and group_unit == 'Kg':
        value = total_qty/1000
    elif by_product_unit_name == 'Gr' and group_unit == 'Nos.':
        value = total_qty
    elif by_product_unit_name == 'Gr' and group_unit == 'Kg':
        value = total_qty
    elif by_product_unit_name == 'Gr' and group_unit == 'Ml':
        value = total_qty/1000
    elif by_product_unit_name == 'Kg' and group_unit == 'Kg':
        value = total_qty
    elif by_product_unit_name == 'Ml' and group_unit == 'L':
        value = total_qty/1000
    elif by_product_unit_name == 'Ml' and group_unit == 'Kg':
        value = total_qty/1000
    elif by_product_unit_name == 'Ml' and group_unit == 'Nos.':
        value = total_qty
    elif by_product_unit_name == 'L' and group_unit == 'Kg':
        value = total_qty
    elif by_product_unit_name == 'Nos.' and group_unit == 'Nos.':
        value = total_qty
    return value

In [2]:
from_date = '2022-07-01'
to_date = '2022-07-31'
business_type_id = 3
import pandas as pd
bysale_abstract_obj = BySale.objects.filter(by_sale_group__ordered_date__range=[from_date, to_date], by_sale_group__business__business_type_id=business_type_id, by_product__short_name='Classic Cone - Cho 100Ml').order_by('by_product__short_name')
bysale_abstract_list = list(bysale_abstract_obj.values_list('by_product_id', 'by_product__by_product_group__hsn_code', 'by_product__short_name', 'count', 'cost', 'by_product__cgst_percent', 'cgst_value', 'by_product__sgst_percent', 'sgst_value', 'by_product__quantity', 'by_product__unit__name','by_product__by_product_group__unit__name', 'total_cost'))
bysale_abstract_column = ["by_product_id","HSN CODE", "PRODUCT", "QTY", "AMOUNT", "CGST%", "CGST AMOUNT", "SGST%", "SGST AMOUNT", "quantity", 'by_product_unit_name', 'group_unit', 'TOTAL AMOUNT']
bysale_abstract_df = pd.DataFrame(bysale_abstract_list, columns=bysale_abstract_column)
bysale_abstract_df = bysale_abstract_df.groupby(['by_product_id', 'PRODUCT']).agg({'HSN CODE':'first', 'QTY':'sum', 'AMOUNT':'sum', 'CGST%':'first', 'CGST AMOUNT':'sum', 'SGST%':'first', 'SGST AMOUNT':'sum', 'quantity': 'first', 'by_product_unit_name': 'first', 'group_unit':'first', 'TOTAL AMOUNT': 'sum'}).reset_index()
# bysale_abstract_df['TOTAL AMOUNT'] = bysale_abstract_df[["AMOUNT", "CGST AMOUNT", "SGST AMOUNT"]].sum(axis=1)

bysale_abstract_df['total_qty'] = bysale_abstract_df['QTY'].astype(float) * bysale_abstract_df['quantity'].astype(float)
bysale_abstract_df['Kg/Ltr'] = bysale_abstract_df.apply(lambda x: to_find_values_in_kg_ltr(x['by_product_unit_name'], x['group_unit'], x['total_qty']), axis=1)

bysale_abstract_df = bysale_abstract_df.drop(['by_product_id'], axis = 1)
# print(bysale_abstract_df['TOTAL AMOUNT'].sum())
bysale_abstract_df.groupby('PRODUCT')['TOTAL AMOUNT'].apply(sum)
bysale_abstract_df



,PRODUCT,HSN CODE,QTY,AMOUNT,CGST%,CGST AMOUNT,SGST%,SGST AMOUNT,quantity,by_product_unit_name,group_unit,TOTAL AMOUNT,total_qty,Kg/Ltr
0,Classic Cone - Cho 100Ml,2105,600.0,15264.0000,9.00,1368.0000,9.00,1368.0000,100.000,Ml,L,18000.0000,60000.0,60.0


In [4]:
by_sale_group_gatepass_obj = BySaleGroupGatepass.objects.filter(bill_date__date__range=[from_date, to_date], by_sale_group__business__business_type_id=business_type_id, by_sale_group__bysale__by_product__short_name='Classic Cone - Cho 100Ml').values_list('by_sale_group__business__businessagentmap__agent__first_name', 'by_sale_group__business__businessagentmap__agent__agent_profile__gst_number', 'by_sale_group__bysale__by_product__short_name','bill_number','bill_date', 'by_sale_group__bysale__sgst_value', 'by_sale_group__bysale__cgst_value', 'by_sale_group__bysale__total_cost', 'by_sale_group__bysale__by_product__by_product_group__hsn_code', 'by_sale_group__bysale__count', 'by_sale_group__bysale__by_product__cgst_percent', 'by_sale_group__bysale__by_product__sgst_percent', 'by_sale_group__bysale__cost')
by_sale_group_gatepass_columns = ["agent_name","gst_number","short_name","bill_number", "bill_date", "sgst_value","cgst_value",  "net_amount", "hsn", 'count', 'cgst', 'sgst', 'cost']
df = pd.DataFrame(list(by_sale_group_gatepass_obj), columns=by_sale_group_gatepass_columns)
df['RATE'] = (df['net_amount'].astype('float')/df['count'].astype('float'))
df['GST'] = df['cgst'].astype('float') + df['sgst'].astype('float')
df['ISGT'] = 0.000
df['bill_date'] = df['bill_date'].dt.strftime('%d-%b-%Y')
df = df.rename(columns={'agent_name':'AGENT', 'gst_number':'BTAXNO', 'short_name':'PRODUCT', 'bill_number':'INVNO', 'bill_date':'INVDATE', 'sgst_value':'SGST','cgst_value':'CGST', 'net_amount':'NETAMOUNT', 'hsn':'HSNCODE'})
final_df_2 = df[['AGENT', 'BTAXNO', 'PRODUCT', 'INVNO', 'INVDATE', 'RATE', 'GST', 'SGST', 'CGST', 'ISGT','NETAMOUNT', 'HSNCODE', 'cost']]
final_df_2 = final_df_2.fillna(' ')
final_df_2.groupby('PRODUCT')['NETAMOUNT'].apply(sum)



PRODUCT
Classic Cone - Cho 100Ml    18000.0000
Name: NETAMOUNT, dtype: object

In [449]:
import pandas as pd
from_date = '2022-04-01'
to_date = '2022-04-04'
business_codes = []
business_types = {}
business_names = []
business_type_list = ['union_booth']
exclude_booth_code = []
option_type = 'by_business_type'
date_list = pd.date_range(start=from_date, end=to_date)
if option_type == 'by_business':
#     business_codes = []
    business_types['business']= business_codes
    business_names = business_codes
elif option_type == 'by_business_type':
    for business_type in business_type_list:
        if business_type == 'private_institute':
            pvt_business_code_list = list(Business.objects.filter(business_type_id=4).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += pvt_business_code_list
            business_types['private_institute'] = pvt_business_code_list
        elif business_type == 'govt_institute':
            gvt_business_code_list = list(Business.objects.filter(business_type_id=5).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += gvt_business_code_list
            business_types['govt_institute'] = gvt_business_code_list
        elif business_type == 'union_parlour':
        # the above codes are used as static for dynamic purpose moved to DB 
            union_parlour_business_code_list = list(BusinessOwnParlourTypeMap.objects.filter(own_parlour_type_id=1).values_list('business__code', flat=True).exclude(business__code__in=exclude_booth_code))
            business_codes += union_parlour_business_code_list
            business_types['union_parlour'] = union_parlour_business_code_list
        elif business_type == 'union_booth':
        #   ['1', '1999', '2000'] 
            union_booth_business_code_list = list(BusinessOwnParlourTypeMap.objects.filter(own_parlour_type_id=2).values_list('business__code', flat=True).exclude(business__code__in=exclude_booth_code))
            business_types['union_booth'] = union_booth_business_code_list
            business_codes += union_booth_business_code_list
        elif business_type == 'nilgris':
            nilgris_booth_business_code_list = list(Business.objects.filter(zone_id=11).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += nilgris_booth_business_code_list
            business_types['nilgris'] = nilgris_booth_business_code_list
        elif business_type == 'tirupur':
            tirupur_booth_business_code_list = list(Business.objects.filter(zone_id=13).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += tirupur_booth_business_code_list
            business_types['tirupur'] = tirupur_booth_business_code_list
        elif business_type == 'society':
            society_business_code_list = list(Business.objects.filter(business_type_id=10).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += society_business_code_list
            business_types['society'] = society_business_code_list
        elif business_type == 'wholesale_delear':
            wsd_business_code_list = list(Business.objects.filter(business_type_id=9).values_list('code', flat=True).exclude(code__in=exclude_booth_code))
            business_codes += wsd_business_code_list
            business_types['wholesale_delear'] = wsd_business_code_list

print(business_codes)
sale_obj = Sale.objects.filter(sale_group__business__code__in=business_codes, sale_group__date__gte=from_date, sale_group__date__lte=to_date, product__group_id=2)
sale_list = list(sale_obj.values_list('id', 'sale_group__business__user_profile_id','sale_group__business_id', 'sale_group__business__code', 'product_id', 'count', 'product__cgst_percent', 'product__sgst_percent', 'cost', 'product__hsn_code'))
sale_column = ['id', 'user_profile_id', 'business_id', 'business_code', 'product_id', 'count', 'cgst_percent', 'sgst_percent' ,'cost', 'hsn_code']
sale_df = pd.DataFrame(sale_list, columns=sale_column)

price_from_trace = serve_product_price_for_date_range(business_codes, from_date, to_date, 2)['return_data_as_dict']

sale_df = sale_df.groupby(['business_id', 'product_id']).agg({'count': sum, 'user_profile_id': 'first', 'business_code': 'first', 'cgst_percent': 'first', 'sgst_percent': 'first', 'cost': sum, 'hsn_code': 'first'}).reset_index()
short_name_and_qty = serve_qty_and_short_name(from_date, to_date)['product_dict']


sale_df['mrp'] = sale_df.apply(lambda x: price_from_trace[x['user_profile_id']][x['product_id']], axis=1)
sale_df['product_quantity'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['qty'], axis=1)
sale_df['product_short_name'] = sale_df.apply(lambda x: short_name_and_qty[x['product_id']]['short_name'], axis=1)
sale_df['base_price'] = sale_df.apply(lambda x: find_base_price_from_gst(x['mrp'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['base_price'] = sale_df['base_price'].astype(float)
sale_df['product_quantity'] = sale_df['product_quantity'].astype(float)
sale_df['cost_by_base_price'] = sale_df['count'] * sale_df['base_price']
sale_df['total_gst_for_base_price'] = sale_df.apply(lambda x: find_the_gst_value(x['cost_by_base_price'], x['cgst_percent'] + x['sgst_percent']), axis=1)
sale_df['qty_in_whole'] = (sale_df['count'] * sale_df['product_quantity']) / 1000
sale_df


['1', '1999', '2000']


,business_id,product_id,count,user_profile_id,business_code,cgst_percent,sgst_percent,cost,hsn_code,mrp,product_quantity,product_short_name,base_price,cost_by_base_price,total_gst_for_base_price,qty_in_whole
0,1714,7,350.0,1725,2000,2.50,2.50,3500.00,0123,10.00,140.0,Curd 140,9.52,3332.00,166.60,49.00
1,1714,25,504.0,1725,2000,6.00,6.00,15120.00,0233,30.00,460.0,Curd 460,26.79,13502.16,1620.26,231.84


In [347]:
def find_the_gst_value(value, total_gst):
    gst = total_gst / 100
    return round(value * float(gst), 2)
    

In [348]:
def find_base_price_from_gst(mrp, total_gst):
    whole_gst = (total_gst / 100) + 1
    base_price = mrp / whole_gst
    return round(base_price, 2)

In [369]:
def serve_qty_and_short_name(start_date, end_date):
    product_dict = {}
    for product in Product.objects.filter():
        if not product.id in product_dict:
            product_dict[product.id] = {}
        trace_ids = []
        for product_trace in ProductNameAndQuantityTrace.objects.filter(product_id=product.id):
            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
            if start_date in date_list :
                trace_ids.append(product_trace.id)
            if end_date in date_list:
                trace_ids.append(product_trace.id)
        if len(set(trace_ids)) == 1:
            product_trace_id = trace_ids[0]
            product_trace_obj = ProductNameAndQuantityTrace.objects.get(id=product_trace_id)
            data_dict = {
                'qty': product_trace_obj.quantity,
                'short_name': product_trace_obj.short_name
            }
            product_dict[product.id] = data_dict
        else:
            data_dict = {
                'is_multiple_qty': True,
            }
            return data_dict
    final_dict = {
        'is_multiple_qty' : False,
        'product_dict': product_dict
    }
    return final_dict

In [40]:
def serve_product_price_for_date_range(business_codes, from_date, to_date, group_id):
    price_per_user_profile_dict = {}
    price_per_user_profile_as_dict_for_gst_bill = {}
    for business_code in business_codes:
        business_obj = Business.objects.get(code=business_code)
        if not business_obj.user_profile.id in price_per_user_profile_dict:
            price_per_user_profile_dict[business_obj.user_profile.id] = []
        if not business_obj.user_profile.id in price_per_user_profile_as_dict_for_gst_bill:
             price_per_user_profile_as_dict_for_gst_bill[business_obj.user_profile.id] = {}
        commission_product_ids = list(BusinessProductConcessionMap.objects.filter(business_type_id=business_obj.business_type_id, product__is_active=True, concession_type_id=1, product__group_id=group_id).values_list('product_id', flat=True))
        discount_product_ids = list(BusinessProductConcessionMap.objects.filter(business_type_id=business_obj.business_type_id, product__is_active=True, concession_type_id=2, product__group_id=group_id).values_list('product_id', flat=True))
        price_per_user_profile_dict[business_obj.user_profile.id] = []
        if len(commission_product_ids) != 0:
            for commission_product_id in commission_product_ids:
                if ProductTrace.objects.filter(product_id=commission_product_id).exists():
                    trace_ids = []
                    for product_trace in ProductTrace.objects.filter(product_id=commission_product_id):
                        date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                        if from_date in date_list :
                            trace_ids.append(product_trace.id)
                        if to_date in date_list:
                            trace_ids.append(product_trace.id)
                    if len(set(trace_ids)) == 1:
                        product_trace_id = trace_ids[0]
                        product_trace_obj = ProductTrace.objects.get(id=product_trace_id)
                        data_dict = {
                            'product_id': product_trace_obj.product.id,
                            'product_mrp': product_trace_obj.mrp
                        }
                        price_per_user_profile_as_dict_for_gst_bill[business_obj.user_profile.id][product_trace_obj.product.id] = product_trace_obj.mrp
                        price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                    else:
                        print(business_obj.code, commission_product_id)
                        data_dict = {
                            'is_multiple_price': True,
                            'return_data': serve_product_diffrence_list(business_obj, 'product_trace', group_id)
                        }
                        return data_dict
        business_type_id = business_obj.business_type_id
        if len(discount_product_ids) != 0:
            if business_type_id == 1 or business_type_id == 2 or business_type_id == 3 or business_type_id == 9 or business_type_id == 11 or business_type_id == 16:  
                for discount_product_id in discount_product_ids:
                    if BusinessTypeWiseProductDiscountTrace.objects.filter(business_type_wise_discount__business_type_id=business_type_id, business_type_wise_discount__product_id=discount_product_id).exists():
                        trace_ids = []
                        for product_trace in BusinessTypeWiseProductDiscountTrace.objects.filter(business_type_wise_discount__business_type_id=business_type_id, business_type_wise_discount__product_id=discount_product_id):
                            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                            if from_date in date_list :
                                trace_ids.append(product_trace.id)
                            if to_date in date_list:
                                trace_ids.append(product_trace.id)
                        
                        if len(set(trace_ids)) == 1:
                            product_trace_id = trace_ids[0]
                            business_type_wise_trace_obj = BusinessTypeWiseProductDiscountTrace.objects.get(id=product_trace_id)
                            data_dict = {
                                'product_id': business_type_wise_trace_obj.business_type_wise_discount.product.id,
                                'product_mrp': business_type_wise_trace_obj.discounted_price
                            }
                            price_per_user_profile_as_dict_for_gst_bill[business_obj.user_profile.id][business_type_wise_trace_obj.business_type_wise_discount.product.id] = business_type_wise_trace_obj.discounted_price
                            
                            price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                        else:
                            print(business_obj.code, discount_product_id)
                            data_dict = {
                                'is_multiple_price': True,
                                'return_data': serve_product_diffrence_list(business_obj, 'business_type_wise', group_id)
                            }
                            return data_dict
            else:
                for discount_product_id in discount_product_ids:
                    if BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id=business_obj.id, business_wise_discount__product_id=discount_product_id).exists():
                        trace_ids = []
                        for product_trace in BusinessWiseProductDiscountTrace.objects.filter(business_wise_discount__business_id=business_obj.id, business_wise_discount__product_id=discount_product_id):
                            date_list = pd.date_range(product_trace.start_date, product_trace.end_date)
                            if from_date in date_list :
                                trace_ids.append(product_trace.id)
                            if to_date in date_list:
                                trace_ids.append(product_trace.id)
                        if len(set(trace_ids)) == 1:
                            product_trace_id = trace_ids[0]
                            business_wise_trace_obj = BusinessWiseProductDiscountTrace.objects.get(id=product_trace_id)
                            data_dict = {
                                'product_id': business_wise_trace_obj.business_wise_discount.product.id,
                                'product_mrp': business_wise_trace_obj.discounted_price
                            }
                            price_per_user_profile_as_dict_for_gst_bill[business_obj.user_profile.id][business_wise_trace_obj.business_wise_discount.product.id] = business_wise_trace_obj.discounted_price

                            price_per_user_profile_dict[business_obj.user_profile.id].append(data_dict)
                        else:
                            print(business_obj.code, discount_product_id)
                            data_dict = {
                                'is_multiple_price': True,
                                'return_data': serve_product_diffrence_list(business_obj, 'business_wise', group_id)
                            }
                            return data_dict
    data_dict = {
        'is_multiple_price': False,
        'return_data': price_per_user_profile_dict,
        'return_data_as_dict': price_per_user_profile_as_dict_for_gst_bill
    }
    return data_dict

In [123]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.display import HTML
import numpy as np
import pandas as pd
from decimal import Decimal
# canvas
import os
from reportlab.pdfgen import canvas
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter, A4, landscape
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from num2words import num2words
from datetime import datetime
import pytz
indian = pytz.timezone('Asia/Kolkata')
from calendar import monthrange, month_name


def data_in_format_function(date_in_string):
    date_in_format = datetime.strptime(str(date_in_string), '%Y-%m-%d')
    date = datetime.strftime(date_in_format, '%d-%m-%y')
    return date

In [453]:
file_name = 'gst_bull' + '.pdf'
file_path = os.path.join('static/media/route_wise_report/', file_name)
last_bill_number_count = InstititionBillNumberIdBank.objects.filter(id=1).first().last_count
bill_no = int(last_bill_number_count) + 1

mycanvas = canvas.Canvas(file_path, pagesize=A4)
light_color = 0x000000
dark_color = 0x000000

for business in business_codes:
    business_obj = Business.objects.get(code=business)
    business_wise_sale_df = sale_df[sale_df['business_id'] == business_obj.id].reset_index()
    if business_wise_sale_df.empty:
        continue
    business_agent_map = BusinessAgentMap.objects.get(business_id=business_obj.id)
    if BusinessWiseBillNumber.objects.filter(from_date=from_date, to_date=to_date, business_id=business_obj.id, product_group_type_id=2).exists():
        temp_bill_number = BusinessWiseBillNumber.objects.filter(from_date=from_date, to_date=to_date, business_id=business_obj.id, product_group_type_id=2).first().bill_number
    else:
        temp_bill_number = bill_no
        business_wise_bill_number = BusinessWiseBillNumber(from_date=from_date, to_date=to_date, business_id=business_obj.id, bill_number=temp_bill_number, created_by_id=1, product_group_type_id=2)
        business_wise_bill_number.save()
        bill_no +=1
    x_a4 = 70
    y_a4 = 25
    mycanvas.setLineWidth(0)

    mycanvas.drawCentredString(300, 820,'Tax Invoice')
    mycanvas.line(15, 815, 570, 815)
    mycanvas.line(15, 610, 15, 815)

    mycanvas.setFont('Helvetica', 11)
    mycanvas.drawString(20, 800, "The Coimbatore Dist. Coop. Milk Producer's union Ltd.")
    mycanvas.drawString(20, 780, 'Pachapalayam, kalampalayam (PO), Coimbatore - 641010')
    mycanvas.drawString(20, 760, 'Ph: 0422-2607971, 2544777')
    mycanvas.drawString(20, 740, 'GSTIN/UIN: 33AAAAT7787L2ZU')
    mycanvas.drawString(20, 720, 'E-Mail: aavincbemkg@gmail.com')
    
    mycanvas.drawString(330, 800, 'Invoice No:')
    mycanvas.drawString(390, 800, str(temp_bill_number))
    mycanvas.drawString(440, 800, 'Date:')
    mycanvas.drawString(480, 800, data_in_format_function(to_date))
    
    
    mycanvas.drawString(330, 780, 'Item supplied period:')
    mycanvas.drawString(450, 780, data_in_format_function(from_date) + ' to ' + data_in_format_function(to_date))
    
    mycanvas.drawString(330, 760, 'Delivery Note:')
    
    mycanvas.drawString(330, 740, 'Despatch Document No:')
    mycanvas.drawString(330, 720, 'Delivery Note Date:')
    mycanvas.line(15, 712, 570, 712)
    mycanvas.line(320, 610, 320, 815)
    # buyer details
    
    address = business_agent_map.agent.agent_profile.street
    pincode = business_agent_map.agent.agent_profile.pincode
    communication_address = business_agent_map.agent.communication_address
    agent_first_name = business_agent_map.agent.first_name
    buyer_string = f"Buyer - {agent_first_name} ( {business_obj.code} )"
    mycanvas.drawString(20, 700, buyer_string)
    
    if communication_address is None:
        communication_address = ""
    if address is None:
        address = ""
    pan_number = business_agent_map.agent.pan_number
    if pan_number is None:
        pan_number = ""
    buyer_gst_number = business_agent_map.agent.agent_profile.gst_number
    if buyer_gst_number is None:
        buyer_gst_number = ""
    if pincode is None:
        pincode = ""

    mycanvas.drawString(20, 680, str(communication_address)+ ' ' + str(address))
    mycanvas.drawString(20, 660, str(pincode))
    
    mycanvas.drawString(20, 640, 'GST No : ' + str(buyer_gst_number))
    mycanvas.drawString(20, 620, 'PAN No : ' + str(pan_number))
    
    
    # route and vehile number
    morning_route = RouteBusinessMap.objects.filter(route__session_id=1, business_id=business_obj.id).first().route
    vehicle_number = RouteVehicleMap.objects.get(route_id=morning_route.id).vehicle.licence_number
    
    mycanvas.drawString(330, 700, 'Despatched through')
    mycanvas.drawCentredString(380, 680, 'PVT Vehicle')
    mycanvas.drawCentredString(380, 660, str(vehicle_number))
    mycanvas.line(470, 712, 470, 610)
    mycanvas.drawString(480, 700, 'Destination')
    mycanvas.drawString(480, 670, str(morning_route.name[:-4]))
    mycanvas.line(15, 610, 570, 610)
    mycanvas.line(570, 610, 570, 815)
    
    x = 50
    y = 580
    mycanvas.line(15, y+10, 575, y+10)
    y_for_start_table = y+10
    # To Write headers
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawCentredString(x-15, y, "S.No")
    mycanvas.drawCentredString(x+40, y, "Description")
    mycanvas.drawCentredString(x+40, y-10, "of Goods")
    mycanvas.drawCentredString(x+105, y, "HSN")
    mycanvas.drawCentredString(x+160, y, "SGST")
    mycanvas.drawCentredString(x+160, y-10, "Rate")
    mycanvas.drawCentredString(x+240, y, "CGST")
    mycanvas.drawCentredString(x+240, y-10, "Rate")
    mycanvas.drawCentredString(x+300, y, "Quantity")
    mycanvas.drawCentredString(x+300, y-10, "(in Bags)")
    mycanvas.drawCentredString(x+360, y,"Quantity")
    mycanvas.drawCentredString(x+360, y-10, "(in Kgs)")
    mycanvas.drawCentredString(x+425, y, "Basic")
    mycanvas.drawCentredString(x+425, y-10, "Rate")
    mycanvas.drawCentredString(x+495, y, "Amount")
    mycanvas.drawCentredString(x+495, y-10, "(in Rs)")
    mycanvas.line(15, y-15, 570, y-15)
    mycanvas.setFont('Helvetica', 10)
    y = y - 30

    total_base_price = 0
    total_sgst_price = 0
    total_cgst_price = 0
    for sale_index, sale in business_wise_sale_df.iterrows():
        mycanvas.drawRightString(x-15, y, str(sale_index+1))
        mycanvas.drawCentredString(x+45, y, str(sale['product_short_name']))
        mycanvas.drawRightString(x+115, y, str(sale['hsn_code']))
        mycanvas.drawRightString(x+175, y, str(sale['sgst_percent']))
        mycanvas.drawRightString(x+255, y, str(sale['cgst_percent']))
        mycanvas.drawRightString(x+315, y, str(int(sale['count'])))
        mycanvas.drawRightString(x+375, y, str('%.2f' % sale['qty_in_whole']))
        mycanvas.drawRightString(x+440, y, str(sale['base_price']))
        mycanvas.drawRightString(x+520, y, str('%.2f' % sale['cost_by_base_price']))
        total_base_price += sale['cost_by_base_price']
        total_sgst_price += sale['total_gst_for_base_price'] / 2
        total_cgst_price += sale['total_gst_for_base_price'] / 2
        y -= 20
    
    total_mrp = total_base_price + total_sgst_price + total_cgst_price
    total_after_round_off = round(total_mrp)
    round_off_value = total_mrp - total_after_round_off
    mycanvas.line(15, y+10, 575, y+10)
    y -= 5
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawString(x+25, y, 'Total')
    mycanvas.drawRightString(x+520, y, str('%.2f' % total_base_price))
    mycanvas.setFont('Helvetica', 10)
    y -= 20
    mycanvas.line(15, y+14, 575, y+14)
    mycanvas.drawString(x+25, y, 'Output SGST')
    mycanvas.drawRightString(x+520, y, str('%.2f' % total_sgst_price))
    y -= 20
    mycanvas.drawString(x+25, y, 'Output CGST')
    mycanvas.drawRightString(x+520, y, str('%.2f' % total_cgst_price))
    y -= 20
    mycanvas.drawString(x+25, y, 'Rounding off')
    mycanvas.drawRightString(x+520, y, str('%.2f' % round_off_value))
    y -= 20
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawString(x+25, y, 'Grand Total')
    mycanvas.drawRightString(x+520, y, str('%.2f' % total_after_round_off))
    mycanvas.setFont('Helvetica', 10)
    mycanvas.line(15, y-5, 575, y-5)
    mycanvas.line(15, y-5, 15, y_for_start_table)
    mycanvas.line(575, y-5, 575, y_for_start_table)
    
    y -= 30
    mycanvas.drawString(20, y, 'Amount Chargeable (in words)')
    y -= 20
    words = num2words(total_after_round_off, lang='en_IN')
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawString(20, y, words.upper())
    mycanvas.setFont('Helvetica', 10)
    y -= 30
    x += 40
    mycanvas.line(15, y+15, 575, y+15)
    y_for_second_table = y + 15
    hsn_wise_df = business_wise_sale_df.groupby('hsn_code').agg({'cgst_percent': 'first',  'sgst_percent': 'first', 'cost_by_base_price': sum, 'total_gst_for_base_price': sum}).reset_index()
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawCentredString(x-60, y, "S.No")
    mycanvas.drawCentredString(x-15, y, "HSN")
    mycanvas.drawCentredString(x+40, y, "Taxable")
    mycanvas.drawCentredString(x+40, y-10, "Value")
    mycanvas.drawCentredString(x+105, y, "SGST")
    mycanvas.drawCentredString(x+105, y-10, "Rate")
    mycanvas.drawCentredString(x+170, y, "SGST")
    mycanvas.drawCentredString(x+170, y-10, "Amount")
    mycanvas.drawCentredString(x+240, y, "CGST")
    mycanvas.drawCentredString(x+240, y-10, "Rate")
    mycanvas.drawCentredString(x+320, y, "CGST")
    mycanvas.drawCentredString(x+320, y-10, "Amount")
    mycanvas.drawCentredString(x+400, y, "Tax")
    mycanvas.drawCentredString(x+400, y-10, "Amount")
    mycanvas.setFont('Helvetica', 10)
    mycanvas.line(15, y-15, 575, y-15)
    y -= 30
    total_taxable = 0
    total_gst = 0
    for index, hsn_wise in hsn_wise_df.iterrows():
        spit_gst = hsn_wise['total_gst_for_base_price'] / 2
        mycanvas.drawCentredString(x-60, y, str(index+1))
        mycanvas.drawCentredString(x-25, y, str(hsn_wise['hsn_code']))
        mycanvas.drawRightString(x+60, y, str('%.2f' % hsn_wise['cost_by_base_price']))
        mycanvas.drawRightString(x+115, y, str(hsn_wise['cgst_percent']))
        mycanvas.drawRightString(x+195, y, str('%.2f' % spit_gst))
        mycanvas.drawRightString(x+255, y, str(hsn_wise['sgst_percent']))
        mycanvas.drawRightString(x+335, y, str('%.2f' % spit_gst))
        mycanvas.drawRightString(x+420, y, str('%.2f' % hsn_wise['total_gst_for_base_price']))
        total_taxable += hsn_wise['cost_by_base_price']
        total_gst += hsn_wise['total_gst_for_base_price']
        y -= 20
        
    y -= 5
    mycanvas.line(15, y+15, 575, y+15)
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawString(x-40, y, 'Total')
    mycanvas.drawRightString(x+60, y, str('%.2f' % total_taxable))
    mycanvas.drawRightString(x+420, y, str('%.2f' % total_gst))
    mycanvas.setFont('Helvetica', 10)
    y -= 30
    mycanvas.line(15, y+20, 575, y+20)
    mycanvas.line(15, y+20, 15, y_for_second_table)
    mycanvas.line(575, y+20, 575, y_for_second_table)
    mycanvas.drawString(20, y, 'Amount Chargeable (in words)')
    y -= 20
    words = num2words(round(total_gst), lang='en_IN')
    mycanvas.setFont('Helvetica-Bold', 10)
    mycanvas.drawString(20, y, words.upper())
    mycanvas.setFont('Helvetica', 10)
    
    
    y -= 30
    mycanvas.drawString(20, 120, "Company's PAN:")
    mycanvas.drawString(120, 120, "AAAAT7787L")
    
    mycanvas.drawString(420, 120, "For C.D.C.M.P.U.Ltd.,")
    mycanvas.drawString(420, 60, "Authorised Signature")
    
    mycanvas.drawString(20, 100, "Declaration")
    mycanvas.drawString(20, 80, "1. The rates are subject to revision from time to time")
    mycanvas.drawString(20, 60, "2. Goods once sold cannot be taken back")
    mycanvas.showPage()
mycanvas.save()
bill_idbank_obj = InstititionBillNumberIdBank.objects.filter(id=1).first()
bill_idbank_obj.last_count = int(bill_no) - 1
bill_idbank_obj.save()



In [1]:
import pandas as pd
import datetime
from decimal import Decimal
from django.db import transaction

from_date = '2022-06-01'
to_date = '2022-08-13'
date_range = pd.date_range(start=from_date, end=to_date)
sid = transaction.savepoint()
try:
    for date in date_range:
        sale_date = str(date)[:10]
        date_in_format = datetime.datetime.strptime(sale_date, '%Y-%m-%d')
        previous_date =  date_in_format - datetime.timedelta(days=1) 
        print(date_in_format)
        GoodsReceiptRecordForDaily.objects.filter(sale_date=date_in_format).delete()
        daily_sale_records = GoodsReceiptRecordForDaily.objects.filter(sale_date=previous_date).exclude(quantity_now=0)
        for daily_sale in daily_sale_records:
            daily_sale_record_obj = GoodsReceiptRecordForDaily(sale_date=date_in_format,
                                                               by_product_id=daily_sale.by_product_id,
                                                               grn_number=daily_sale.grn_number,
                                                               quantity_at_receipt=daily_sale.quantity_now,
                                                               quantity_now=daily_sale.quantity_now,
                                                               quantity_now_time=datetime.datetime.now(),
                                                               price_per_unit=daily_sale.price_per_unit,
                                                               total_price=daily_sale.total_price,
                                                               )
            daily_sale_record_obj.save()
            print('old sale updated')
        if GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__date=date_in_format).exists():
            print('goods receipt available')
            goods_receipt_record_objs = GoodsReceiptRecord.objects.filter(goods_receipt_master__grn_date__date=date_in_format)
            for goods_receipt_record_obj in goods_receipt_record_objs:
                daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily(sale_date=date_in_format,
                                                              by_product_id=goods_receipt_record_obj.by_product.id,
                                                              quantity_at_receipt=goods_receipt_record_obj.quantity_at_receipt,
                                                              quantity_now=goods_receipt_record_obj.quantity_at_receipt,
                                                              quantity_now_time=goods_receipt_record_obj.quantity_now_time,
                                                              price_per_unit=goods_receipt_record_obj.price_per_unit,
                                                              total_price=goods_receipt_record_obj.total_price,
                                                              grn_number=goods_receipt_record_obj.goods_receipt_master.grn_number,
                                                              )
                daily_goods_receipt_record_obj.save()
        by_sale_objs = BySale.objects.filter(by_sale_group__ordered_date=date_in_format)
        for sale in by_sale_objs:
            print(sale.count)
            daily_goods_receipt_record_obj = GoodsReceiptRecordForDaily.objects.filter(by_product_id=sale.by_product.id, sale_date=date_in_format).exclude(quantity_now=0).order_by('id')
            required_qty = sale.count
            for goods in daily_goods_receipt_record_obj:
                if goods.quantity_now >= required_qty:
                    balance_in_goods = goods.quantity_now - Decimal(required_qty)
                    goods.quantity_now = balance_in_goods
                    goods.quantity_now_time = datetime.datetime.now()
                    goods.save()

                    goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                        daily_goods_receipt_record_id=goods.id,
                        by_sale_id=sale.id,
                        quantity_dispatched=required_qty
                    )
                    goods_receipt_record_sale_map_obj.save()
                    break
                else:
                    required_qty = Decimal(required_qty) - goods.quantity_now
                    goods_receipt_record_sale_map_obj = DailyGoodsReceiptRecordBySaleMap(
                        daily_goods_receipt_record_id=goods.id,
                        by_sale_id=sale.id,
                        quantity_dispatched=goods.quantity_now
                    )
                    goods_receipt_record_sale_map_obj.save()

                    goods.quantity_now = 0
                    goods.quantity_now_time = datetime.datetime.now()
                    goods.save()
    transaction.savepoint_commit(sid)
except Exception as e:
    print('Error on {}'.format(e))
    transaction.savepoint_rollback(sid)

2022-06-01 00:00:00
2022-06-02 00:00:00
2022-06-03 00:00:00
2022-06-04 00:00:00
2022-06-05 00:00:00
1.0
1.0
2.0
2022-06-06 00:00:00
2022-06-07 00:00:00
2022-06-08 00:00:00
2022-06-09 00:00:00
2022-06-10 00:00:00


In [9]:
from decimal import Decimal
for by_product in ByProduct.objects.filter():
    in_stock = 0
    in_receipt = 0
    if GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).exists():
        in_stock = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_now'))['quantity_now__sum']
        in_receipt = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).aggregate(Sum('quantity_at_receipt'))['quantity_at_receipt__sum']
    orderable_qty = 0
    total_order = 0
    if BySale.objects.filter(by_product_id=by_product.id).exists():
        total_order = BySale.objects.filter(by_product_id=by_product.id).aggregate(Sum('count'))['count__sum']
    if ByProductCurrentAvailablity.objects.filter(by_product_id=by_product.id).exists():
        orderable_qty = ByProductCurrentAvailablity.objects.get(by_product_id=by_product.id).quantity_now
    difference = orderable_qty - in_stock
    dif = Decimal(in_receipt) - Decimal(total_order)
    
    if difference != 0:
        print(f"issue at {by_product.short_name} \t {difference} \t {orderable_qty} \t {in_stock} \t {by_product.id} \t {in_receipt} \t {total_order}")
#         print(orderable_qty, dif, difference)
#         latest_entry = GoodsReceiptRecord.objects.filter(by_product_id=by_product.id).last()
#         latest_entry.quantity_now += difference
#         latest_entry.save()
#         print(latest_entry.quantity_now)
#     if difference > 0 :
#         print('sale to be added', difference, orderable_qty, in_stock, by_product.id)


In [ ]:
GoodsReceiptRecord.objects.filter(by_product_id=by_product.id)